<a href="https://colab.research.google.com/github/GenghisKhandybar/Movie-Data-Analysis/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
movies = []

# For the first n movies
for i in range(5000):

  # Create a list of those n movies
  response = requests.get("https://api.themoviedb.org/3/movie/" + str(i) + "?api_key=bb05b1937aaac2e718a24f9c1b42d2fb")
  movies += [response.json()]
  time.sleep(0.5)

In [ ]:
# Create a dataframe from those movies
data_movies = json_normalize(movies)

# Remove any bad requests from the API
df_movies = data_movies[data_movies["success"] != False]

# Remove some unnecessary columns
df_movies.drop(axis=1, columns = ["success", "status_code", "status_message"], inplace=True)
# Reset the index
df_movies.reset_index(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Create a function to map genre JSON to a more accessible data format
def genre_mapper(response):
  genre_list = []
  for i in range(len(response)):
    genre_list += [response[i]['name']]
  return genre_list

# Map genres to accessible format
df_movies["genres"] = df_movies["genres"].map(genre_mapper)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
# Create a dataframe of the ratings from imdb
df_ratings = pd.read_csv("imdb_ratings.txt", sep="\t")

In [ ]:
# Create a dataframe of people from imdb
df_people = pd.read_csv("imdb_names.txt", sep="\t")

# Keep only necessary columns
df_people = df_people[["nconst", "primaryName"]]

In [ ]:
# Create a dataframe of the crew from imdb
df_crew = pd.read_csv("imdb_crews.txt", sep="\t")

# Drop unnecessary columns
df_crew.drop(axis=1, columns=["writers"], inplace=True)

In [ ]:
# Merge datasets to get a list of directors for each movie
data_crew_names = df_crew.merge(df_people, left_on="directors", right_on="nconst")
data_crew_names.rename(columns={"primaryName":"director"}, inplace=True)
data_crew_names.drop(axis=1, columns=["directors", "nconst"], inplace=True)

In [ ]:
# Create a dataset where we merge the data obtained from the API with the ratings from IMDB
df_combined = df_movies.merge(df_ratings, left_on="imdb_id", right_on="tconst")

In [ ]:
# Create the final dataset by merging directors with movies and ratings
df_movies = df_combined.merge(data_crew_names, on="tconst")

# Remove all unnecessary columns
df_movies.drop(axis=1, 
               columns=["index", "tconst", "poster_path", "homepage", "backdrop_path", "vote_average", "vote_count"], inplace=True)

In [ ]:
#Adding variables for plots
df_movies["ROI"] = df_movies["revenue"]/df_movies["budget"] 
df_movies["language"] = df_movies["original_language"].map(lambda l: "English" if l=="en" else "Not English")

#Creating dataset for director related plot
df_director_stats = df_movies[df_movies["budget"]>0].groupby("director").mean().drop("Nicolas Winding Refn")
df_director_stats["movies"] = df_movies[df_movies["budget"]>0]["director"].value_counts()

In [ ]:
series = df_movies["belongs_to_collection.id"].value_counts().index.tolist() #A list of all the "series IDs"
df_all_series = pd.DataFrame()

for s in series: #For all series that we found one of, we get the listing for every movie in that series.
  response = requests.get("https://api.themoviedb.org/3/collection/" + str(s) + "?api_key=bb05b1937aaac2e718a24f9c1b42d2fb&language=en-US")
  df_s = json_normalize(response.json()["parts"])
  df_s["collection_order"] = df_s.index + 1 #So we start with movie number 1 instead of 0
  df_all_series = df_all_series.append(df_s)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [ ]:
df_all_series = df_all_series[df_all_series["vote_count"] > 0].drop(["video"], axis=1)
df_full_series = df_all_series.merge(df_movies, on="id", how="right") 
#Right merge because we need all the movies with no series, but can't use series movies without the data from df_movies.
df_full_series["collection_order"] = df_full_series["collection_order"].fillna(0) #Standalone movies are considered 0 in series.

df_full_series["ROI"] = df_full_series["revenue"]/df_full_series["budget"]
df_full_series["made_even"] = df_full_series["ROI"].apply(lambda x: "Profitable" if x>1 else "Unprofitable")
df_full_series["Standalone"]  = df_full_series["collection_order"].map(lambda x: "Standalone" if x==0 else "In a Series")
df_full_series = df_full_series[df_full_series["budget"]>1000] 
#0 budget and very small budget films would mess up the presentation of data, and are most likely wrong as making a movie is expensive.

In [ ]:
from google.colab import files
df_full_series.to_csv('df_full_series.csv')
df_director_stats.to_csv('df_directors.csv')
df_movies.to_csv('df_movies.csv')

files.download("df_full_series.csv")
files.download('df_directors.csv')
files.download('df_movies.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>